In [ ]:
import numpy as np
import tables_io

In [ ]:
def split_by_type_and_clean(basefile, types=['ns', '1m', '2m', '1p', '2p'], shear=0.01):
    p = tables_io.read(basefile)
    for type_ in types:
        mask = p['shear_type'] == type_
        sub = p[mask]
        idx_x = 20*sub['patch_x'].values + sub['cell_x'].values
        idx_y = 20*sub['patch_y'].values + sub['cell_y'].values
        cent_x = 150*idx_x -75
        cent_y = 150*idx_y -75
        local_x = sub['col'] - cent_x
        local_y = sub['row'] - cent_y
        central_to_cell = np.bitwise_and(np.fabs(local_x) < 75, np.fabs(local_y) < 75)
        central_to_patch = np.bitwise_and(np.fabs(sub['cell_x'].values - 10.5) < 10, np.fabs(sub['cell_y'].values - 10.5) < 10)
        central = np.bitwise_and(central_to_cell, central_to_patch)
        cleaned = sub[central]
        cleaned['shear'] = np.repeat(shear, len(cleaned))
        cleaned.to_parquet(basefile.replace('.parq', f"_cleaned_{type_}.pq"))

In [ ]:
shear_value_strs = ['0p0025', '0p005', '0p01', '0p02', '0p04']
shear_values = [0.0025, 0.005, 0.01, 0.02, 0.04]

In [ ]:
for shear_st_, shear_ in zip(shear_value_strs, shear_values):
    split_by_type_and_clean(f"shear_tract/shear_wmom_{shear_st_}.parq", shear=shear_)